<a href="https://colab.research.google.com/github/greyhound101/IEEE-CIS-Fraud/blob/master/raw.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
pip install tensorflow==2.1.0-rc0

     |████████████████████████████████| 402.3MB 34kB/s 
     |████████████████████████████████| 3.8MB 53.8MB/s 
     |████████████████████████████████| 450kB 48.3MB/s 
  Created wheel for gast: filename=gast-0.2.2-cp36-none-any.whl size=7540 sha256=082e643d7e370acf1406227dc063d11dbdfe3f20255cca5772ad7d0b4b6f6fd9
  Stored in directory: /root/.cache/pip/wheels/5c/2e/7e/a1d4d4fcebe6c381f378ce7743a3ced3699feb89bcfbdadadd
Successfully built gast
ERROR: tensorflow-probability 0.10.0rc0 has requirement gast>=0.3.2, but you'll have gast 0.2.2 which is incompatible.
  Found existing installation: tensorboard 2.2.1
    Uninstalling tensorboard-2.2.1:
      Successfully uninstalled tensorboard-2.2.1
  Found existing installation: gast 0.3.3
    Uninstalling gast-0.3.3:
      Successfully uninstalled gast-0.3.3
  Found existing installation: tensorflow-estimator 2.2.0
    Uninstalling tensorflow-estimator-2.2.0:
      Successfully uninstalled tensorflow-estimator-2.2.0
  Found existing installatio

In [2]:
pip install keras==2.3.1

In [9]:
import os
os.environ['KAGGLE_USERNAME'] = "tapaskd123" # username from the json file
os.environ['KAGGLE_KEY'] = "aba8dc1f085221111d925003fe5a88ed" # key from the json file
!kaggle competitions download -c ieee-fraud-detection

 79% 41.0M/52.2M [00:00<00:00, 84.8MB/s]
100% 52.2M/52.2M [00:00<00:00, 132MB/s] 
  0% 0.00/3.21M [00:00<?, ?B/s]
100% 3.21M/3.21M [00:00<00:00, 217MB/s]
  0% 0.00/1.14M [00:00<?, ?B/s]
100% 1.14M/1.14M [00:00<00:00, 160MB/s]
 57% 33.0M/58.3M [00:00<00:00, 64.3MB/s]
100% 58.3M/58.3M [00:00<00:00, 168MB/s] 
  0% 0.00/3.26M [00:00<?, ?B/s]
100% 3.26M/3.26M [00:00<00:00, 218MB/s]


In [4]:

from google.colab import drive
drive.mount('/content/gdrive')
drive.mount("/content/gdrive", force_remount=True)

Go to this URL in a browser: https://accounts.google.com/o/oauth2/auth?client_id=947318989803-6bn6qk8qdgf4n4g3pfee6491hc0brc4i.apps.googleusercontent.com&redirect_uri=urn%3aietf%3awg%3aoauth%3a2.0%3aoob&response_type=code&scope=email%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdocs.test%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive.photos.readonly%20https%3a%2f%2fwww.googleapis.com%2fauth%2fpeopleapi.readonly

Enter your authorization code:
··········
Mounted at /content/gdrive
Mounted at /content/gdrive


In [10]:
import numpy as np
import pandas as pd
#read raw data
trn_trans=pd.read_csv('train_identity.csv.zip')
trn=pd.read_csv('train_transaction.csv.zip')
tst_trans=pd.read_csv('test_identity.csv.zip')
tst=pd.read_csv('test_transaction.csv.zip')

#merge data

fnl_trn=trn.merge(trn_trans,on='TransactionID',how='left')
fnl_tst=tst.merge(tst_trans,on='TransactionID',how='left')

#drop columns causing overfit

fnl_trn=fnl_trn.drop(list(fnl_trn.filter(regex='V')),1)
fnl_tst=fnl_tst.drop(list(fnl_tst.filter(regex='V')),1)

#rename columns

import gc
dk={}
for i in range(1,39):
    if i<10:
        dk['id-0'+str(i)]='id_0'+str(i)
    else:
        dk['id-'+str(i)]='id_'+str(i)
fnl_tst=fnl_tst.rename(columns=dk)

#free memory

del([trn,tst,trn_trans,tst_trans])
gc.collect()

0

In [11]:
#give identifier column

fnl_trn['is_train']=1
fnl_tst['is_train']=0

#concat all data

fnl=pd.concat([fnl_trn,fnl_tst],0)
fnl=fnl.reset_index(drop=True)
gc.collect()
numerical = ["TransactionAmt", "dist1", "dist2"] + ["C" + str(i) for i in range(1, 15)]+["D" + str(i) for i in range(1, 16)]

categorical = ["ProductCD", "card1", "card2", "card3", "card4", "card5", "card6", "addr1", "addr2",
               "P_emaildomain", "R_emaildomain",
              "DeviceInfo", "DeviceType"] + ["id_0" + str(i) for i in range(1, 10)] +\
                ["id_" + str(i) for i in range(10, 39)] + \
                 ["M" + str(i) for i in range(1, 10)]

#fill numericals with their mean and categoricals with nan
fnl[["D" + str(i) for i in range(1, 16)]]=fnl[["D" + str(i) for i in range(1, 16)]].fillna(-1)
fnl[["C" + str(i) for i in range(1, 15)]]=fnl[["C" + str(i) for i in range(1, 15)]].fillna(-1)
for col in numerical:
    fnl[col]=fnl[col].fillna(fnl[col].mean())
for col in categorical:
    fnl[col]=fnl[col].fillna('nan')

    
#create id column    

fnl['days']=fnl['TransactionDT']//86400
fnl['id']=fnl['days']-fnl['D1']
fnl['id']=fnl['id'].astype(str)+fnl['P_emaildomain'].astype(str)+fnl['card1'].astype(str)+fnl['C9'].astype(str)+fnl['C11'].astype(str)+fnl['dist1'].astype(str)
from sklearn.preprocessing import LabelEncoder
le=LabelEncoder()
fnl['id']=le.fit_transform(fnl['id'])

#for numericals groupby id then find mean and std

from tqdm import tqdm
for col in tqdm(numerical):
    try:
        fnl[col+'_mean']=fnl.groupby(['id'])[col].transform('mean')
        fnl[col+'_std']=fnl.groupby(['id'])[col].transform('std')
    except:
        continue
fnl=fnl.drop(['id'],1)

#get dummies of top 5 most frequent values for categoricals

from tqdm import tqdm
for col in tqdm(categorical):
        print('before '+str(fnl[col].nunique()))
        fnl.loc[~fnl[col].isin(list(fnl[col].value_counts().index[:5])),col]='other'
        print('before '+str(fnl[col].nunique()))
        if col!='card1' or col !='id_02':
            fnl_dum=pd.get_dummies(fnl[col]).astype('uint8')
            fnl_dum.columns=list(np.char.add(col,np.asarray(list(fnl_dum)).astype(str)))
            fnl_dum.columns=fnl_dum.columns+'_dum'
            
            #fnl=fnl.drop([col],1)
            fnl=pd.concat([fnl,fnl_dum],1)
fnl['day']=fnl['TransactionDT']//86400
fnl['month']=fnl['day']//30
fnl['week']=fnl['day']//7

  0%|          | 0/60 [00:00<?, ?it/s]

before 5
before 5


  2%|▏         | 1/60 [00:01<01:24,  1.43s/it]

before 17091
before 6


  3%|▎         | 2/60 [00:04<02:00,  2.07s/it]

...

 97%|█████████▋| 58/60 [02:45<00:05,  2.88s/it]

before 3
before 3


 98%|█████████▊| 59/60 [02:48<00:02,  2.89s/it]

before 3
before 3


100%|██████████| 60/60 [02:50<00:00,  2.85s/it]


In [0]:
import lightgbm as lgb
import gc
fnl.columns = ["".join (c if c.isalnum() else "_" for c in str(x)) for x in fnl.columns]
fnl=fnl.drop(['TransactionID','TransactionDT'],1)
fnl=fnl.reset_index(drop=True)
gc.collect()
fnl=fnl.fillna(-1)
def reduce_mem_usage(df):
    """ iterate through all the columns of a dataframe and modify the data type
        to reduce memory usage.        
    """
    start_mem = df.memory_usage().sum() / 1024**2
    print('Memory usage of dataframe is {:.2f} MB'.format(start_mem))
    
    for col in df.columns:
        col_type = df[col].dtype
        if col_type in ['int','float']:
            df[col]=df[col].fillna(-1)
            df[col]=df[col].replace([np.inf,-np.inf],-1)
        
            c_min = df[col].min()
            c_max = df[col].max()
            if str(col_type)[:3] == 'int':
                if c_min > np.iinfo(np.int8).min and c_max < np.iinfo(np.int8).max:
                    df[col] = df[col].astype(np.int8)
                elif c_min > np.iinfo(np.int16).min and c_max < np.iinfo(np.int16).max:
                    df[col] = df[col].astype(np.int16)
                elif c_min > np.iinfo(np.int32).min and c_max < np.iinfo(np.int32).max:
                    df[col] = df[col].astype(np.int32)
                elif c_min > np.iinfo(np.int64).min and c_max < np.iinfo(np.int64).max:
                    df[col] = df[col].astype(np.int64)  
            else:
                if c_min > np.finfo(np.float16).min and c_max < np.finfo(np.float16).max:
                    df[col] = df[col].astype(np.float16)
                elif c_min > np.finfo(np.float32).min and c_max < np.finfo(np.float32).max:
                    df[col] = df[col].astype(np.float32)
                else:
                    df[col] = df[col].astype(np.float64)
        else:
            df[col] = df[col].astype('category')

    end_mem = df.memory_usage().sum() / 1024**2
    print('Memory usage after optimization is: {:.2f} MB'.format(end_mem))
    print('Decreased by {:.1f}%'.format(100 * (start_mem - end_mem) / start_mem))
    
    return df

#add fraud column

trn=pd.read_csv('train_transaction.csv.zip',usecols=['isFraud'])
fnl['isFraud']=trn['isFraud']
fnl=fnl.loc[:,~(fnl.columns.duplicated())]

In [13]:
from tqdm import tqdm
import gc
pds=['day','month','week']

#since these columns increase with time scale these columns wrt different time periods 

cols =  ["C" + str(i) for i in range(1, 15)] + \
            ["D" + str(i) for i in range(1, 16)] 
for pd in pds:
  for col in tqdm(cols):
    col=str(col)
    use_col=str(col)+'_'+pd+'_ss'
    fnl['a']=fnl.groupby([pd])[col].transform('max')
    fnl['b']=fnl.groupby([pd])[col].transform('min')
    fnl[use_col]=fnl[col]-fnl['b']/(fnl['a']-fnl['b'])
    use_col=str(col)+'_'+pd+'_norm'
    fnl['a']=fnl.groupby([pd])[col].transform('std')
    fnl['b']=fnl.groupby([pd])[col].transform('mean')
    fnl[use_col]=(fnl[col]-fnl['a'])/fnl['b']
  
del([fnl['a'],fnl['b']])
gc.collect()

100%|██████████| 29/29 [00:58<00:00,  2.02s/it]


0

In [14]:

#replace inf with 0 and remove duplicate columns then reduce memory

fnl=fnl.replace([np.inf,-np.inf],0)
fnl=fnl.loc[:,~(fnl.columns.duplicated())]
fnl=reduce_mem_usage(fnl)

Memory usage of dataframe is 3122.46 MB
Memory usage after optimization is: 947.03 MB
Decreased by 69.7%


In [15]:

from google.colab import drive
drive.mount('/content/gdrive')
drive.mount("/content/gdrive", force_remount=True)
path = F"/content/gdrive/My Drive/raw.csv" 
fnl.to_csv(path)

Drive already mounted at /content/gdrive; to attempt to forcibly remount, call drive.mount("/content/gdrive", force_remount=True).
Mounted at /content/gdrive
